<a href="https://colab.research.google.com/github/srijit43/NLP-theory-and-code/blob/main/05_06_Exploring_RandomForestModel_with_Gridsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Machine Learning Classifiers: Explore Random Forest model with grid-search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

### Read in & clean text

In [2]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Adjustment

In [3]:
X_features = X_features.rename(str,axis='columns')

### Build our own Grid-search

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

###We will build a nested for loop to test our grid search

In [8]:
def train_RF(n_est,n_depth):
  rf = RandomForestClassifier(n_estimators = n_est,max_depth = n_depth,n_jobs = -1)
  rf_model = rf.fit(X_train,y_train)
  y_pred = rf_model.predict(X_test)
  precision,recall,fscore,support = score(y_test,y_pred,average = 'binary',pos_label = 'spam')
  print("Est: {} / Depth:{} --- > Precision {},Recall {},Accuracy {}".format(n_est,n_depth,round(precision,3),round(recall,3),round((y_pred==y_test).sum()/len(y_test),3)))



###Now we are going to plug in

In [9]:
for n_est in [10,50,100]:
  for n_depth in [10,20,30,None]:
    train_RF(n_est,n_depth)

Est: 10 / Depth:10 --- > Precision 1.0,Recall 0.35,Accuracy 0.917
Est: 10 / Depth:20 --- > Precision 1.0,Recall 0.629,Accuracy 0.952
Est: 10 / Depth:30 --- > Precision 0.99,Recall 0.706,Accuracy 0.961
Est: 10 / Depth:None --- > Precision 1.0,Recall 0.755,Accuracy 0.969
Est: 50 / Depth:10 --- > Precision 1.0,Recall 0.245,Accuracy 0.903
Est: 50 / Depth:20 --- > Precision 1.0,Recall 0.636,Accuracy 0.953
Est: 50 / Depth:30 --- > Precision 1.0,Recall 0.72,Accuracy 0.964
Est: 50 / Depth:None --- > Precision 1.0,Recall 0.783,Accuracy 0.972
Est: 100 / Depth:10 --- > Precision 1.0,Recall 0.259,Accuracy 0.905
Est: 100 / Depth:20 --- > Precision 1.0,Recall 0.594,Accuracy 0.948
Est: 100 / Depth:30 --- > Precision 1.0,Recall 0.685,Accuracy 0.96
Est: 100 / Depth:None --- > Precision 1.0,Recall 0.811,Accuracy 0.976
